In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.morphology import *

In [ ]:
def writeBOV(fname_base, byte_blk, var_name):
    reorder_array(byte_blk).tofile(fname_base+'.bytes')
    scan_sz = byte_blk.shape
    with open(fname_base + '.bov', 'w') as f:
        f.write("TIME: 0\n")
        f.write("DATA_FILE: %s\n" % (fname_base + '.bytes'))
        f.write("DATA_SIZE: %d %d %d\n" % (scan_sz[0], scan_sz[1], scan_sz[2]))
        f.write("DATA_FORMAT: BYTE\n")
        f.write("VARIABLE: %s\n" % var_name)
        f.write("DATA_ENDIAN: LITTLE\n")
        f.write("CENTERING: ZONAL\n")
        f.write("BRICK_ORIGIN: 0.0 0.0 0.0\n")
#        f.write("BRICK_ORIGIN: %f %f %f\n" % (float(x_base + 4750), float(y_base + 2150),                                                                     
#                                              float(z_start_offset + z_base + 4000)))                                                                         

        f.write("BRICK_SIZE: %f %f %f\n" % (float(scan_sz[0]), float(scan_sz[1]), float(scan_sz[2])))


In [ ]:
def reorder_array(blk):
    """
    Operations to flip the array orders to match the original data
    samples
    """
    return np.flip(blk.transpose(), 1)


In [ ]:
class BlockData(object):
    def __init__(self, bovname):
        print(bovname)
        data_dir = os.path.dirname(bovname)
        words = os.path.splitext(os.path.basename(bovname))[0].split('_')
        self.grpnm = words[0]
        self.offset = np.asarray([int(x) for x in words[2:5]], dtype=np.int)
        with open(bovname, 'r') as f:
            for line in f.readlines():
                key, parm_str = line.split(':')
                parms = [elt.strip() for elt in parm_str.split()]
                if key == 'DATA_FILE':
                    self.data_file = parms[0]
                elif key == 'DATA_SIZE':
                    self.shape = np.asarray([int(x) for x in parms], dtype=np.int)
                elif key == 'VARIABLE':
                    self.var = parms[0]
                elif key == 'DATA_FORMAT':
                    assert len(parms) == 1 and parms[0] == 'BYTE', 'Only byte data is currently supported'
                elif key == 'CENTERING':
                    assert len(parms) == 1 and parms[0] == 'ZONAL', 'Only zonal data is currently supported'
        data = np.fromfile(os.path.join(data_dir, self.data_file),
                           dtype=np.uint8).reshape(*self.shape)
        
        self.data = reorder_array(data)
        #self.data = data
                    
    def __repr__(self):
        return "BlockData<{}({}, {})>".format(self.var, str(self.offset), str(self.shape))

In [ ]:
predNameL = glob.glob('/home/welling/sshfshook/mastercube*pred*.bov')
scannedNameL = glob.glob('/home/welling/sshfshook/mastercube*scanned*.bov')


In [ ]:
pred_blks = [BlockData(nm) for nm in predNameL]
scanned_blks = [BlockData(nm) for nm in scannedNameL]


In [ ]:
def merge_blocks(blks):
    llcV = blks[0].offset.copy()
    trbV = blks[0].offset.copy() + blks[0].shape.copy()
    for blk in pred_blks[1:]:
        llcV = np.minimum(llcV, blk.offset)
        trbV = np.maximum(trbV, blk.offset + blk.shape)
    shape_sz_V = trbV - llcV
    #print('llc: ',llcV)
    #print('trb: ',trbV)
    #print('shape_sz_V', shape_sz_V)
    space = np.zeros(shape_sz_V, dtype=np.uint8)
    for blk in blks:
        local_llcV = blk.offset - llcV
        #print('offset: ',blk.offset, 'local_llcV: ', local_llcV)
        local_trbV = local_llcV + blk.shape
        space[local_llcV[0]:local_trbV[0], local_llcV[1]:local_trbV[1], local_llcV[2]:local_trbV[2]] = blk.data
    return space
    

In [ ]:
scanned_merged = merge_blocks(scanned_blks)
writeBOV('test_scanned', scanned_merged, scanned_blks[0].var)

pred_merged = merge_blocks(pred_blks)
strct = generate_binary_structure(3, 3)
pred_morph = binary_opening(pred_merged, iterations=2)
writeBOV('test_pred', pred_morph, pred_blks[0].var)
